In [1]:
from __future__ import division
import numpy as np
import pickle
%matplotlib inline
import matplotlib.pyplot as plt
import math
import lmdb
import itertools
import json

# Make sure that caffe is on the python path:
caffe_root = '../../'  # this file is expected to be in {caffe_root}/examples/language_model
import sys
sys.path.insert(0, caffe_root + 'python')
import apollocaffe
sys.path.insert(0, caffe_root + 'python/apollocaffe/proto')
import caffe_pb2
sys.path.insert(0, caffe_root + 'examples/english_input_predictor4')
from train import (load_indice_file, forward)

In [2]:
config = json.load(open("config.json", 'r'))
net_config = config["net"]
apollocaffe.set_random_seed(config["solver"]["random_seed"])
apollocaffe.set_device(0)

2015-12-28 08:59:11 - GPU device 0


In [3]:
# Warning: load_idl returns an infinite generator. Calling list() before islice() will hang.
test_gen = load_indice_file(config["data"]["test_file"], config["net"])

In [4]:
net = apollocaffe.ApolloNet()
net.phase = 'train'
forward(net, test_gen.next(), config["net"], False)
net.load("/home/pig/apollocaffe-master/examples/english_input_predictor4/tmp/reinspect_180000.h5")

/home/pig/apollocaffe-master/data/english_input_predictor/test_indices.txt 98959


In [5]:
vocab = pickle.load(open(caffe_root + 'data/english_input_predictor/vocab.pkl', 'rb'))
inv = lambda d: {v:k for k,v in d.iteritems()}
vocab_inv = inv(vocab)

In [10]:
def indice_2_string(indice):
    string = ''
    for i in indice:
        i = int(i)
        if i == net_config['unknown_symbol']:
            string += '#'
        elif i == net_config['start_symbol']:
            string += '$'
        elif i == net_config['zero_symbol']:
            string += ' '
        elif i==4:
            string += '#'
        else:
            string += vocab_inv[i]
    return string
def get_max_indice(probs):
    indice = []
    for i in range(net_config['max_len']):
        peakIndex = np.argmax(probs[i,:])
        indice.append(peakIndex)
    return indice
def if_the_same(pred_indice, target_indice):
    for i in range(len(target_indice)):
        if target_indice[i]!=net_config['zero_symbol'] and target_indice[i]!=pred_indice[i]:
            return False
    return True

In [7]:
test_gen = load_indice_file(config["data"]["test_file"], config["net"], phase = 'test')

In [18]:
test_en = test_gen.next()
forward(net, test_en, config["net"], phase = 'test', deploy = True)
print net.loss

0.743317365646


In [26]:
test_en = test_gen.next()
forward(net, test_en, config["net"], phase = 'test', deploy = True)
print net.loss
accuracy = 0
for batch_id in range(net_config['batch_size']):
    data = test_en["input_words"]
    input_indice = data[batch_id,:,0,0]

    data = test_en["target_words"]
    target_indice = data[batch_id,:,0,0]

    probs = net.blobs['word_probs'].data
    out_prob = np.zeros((net_config['max_len'], net_config['vocab_size']))
    for i in range(net_config['max_len']):
        out_prob[i,:] = probs[batch_id+i*net_config['batch_size'],:]
    output_indice = get_max_indice(out_prob)

    out_prob = np.zeros((net_config['max_len'], net_config['vocab_size']))
    for i in range(net_config['max_len']):
        out_prob[i,:] = net.blobs['target_wordvec%d'%i].data[batch_id,:]
    input_indice2 = get_max_indice(out_prob)
    
    if if_the_same(output_indice, target_indice):
        accuracy += 1
    else:
        pass
        #print indice_2_string(input_indice)
        #print indice_2_string(output_indice)
        #print indice_2_string(target_indice)
print accuracy/net_config['batch_size']

0.671631753445
0.8359375


In [ ]:
print vocab